# SQL Common Table Expressions (CTE)

In [ ]:
!wget https://github.com/gt-cse-6040/bootcamp/raw/main/SQL/syllabus/NYC-311-2M_small.db

In [ ]:
# create a connection to the database
import sqlite3 as db
import pandas as pd

# Connect to a database (or create one if it doesn't exist)
conn_nyc = db.connect('NYC-311-2M_small.db')

## Common Table Expressions (CTEs) -- Quick Review

A Common Table Expression (CTE), introduced by the WITH clause, is a named temporary result set that you can reference within your main query. 

CTEs improve the readability and maintainability of SQL code, especially for complex queries, by allowing you to break down operations into logical steps.

CTEs are often preferred for their clarity and efficiency, particularly when the same calculation or transformation needs to be referenced multiple times within a query. 

Visit the [SQLite Documentation's CTE page](https://www.sqlite.org/lang_with.html) to learn more.

## CTEs

- Are created/defined at the `top` of the SQL statement.

- Are called, as a separate operation, each time that they are referenced.

- As such, are reusable.

- What this means, practically, is that if you are joining to a CTE multiple times in your SQL, it is separately called and executed each time you reference it.

### EXAMPLE -- INNER JOIN WITH CTE (same as before, with a subquery)

**Requirement**

From the `data` table, for each `city`, return counts or a distribution of tickets per `hour` on the biggest day (by events) by `createdDate`. 

Hint, it's `2014-11-18` (8466 events) but how do we put this into code dynamically?

*    Columns
    *    `City`
    *    `createdHour`
    *    `countoccur`: the count of events

*    Exclude NULL cities i.e. `WHERE city IS NOT NULL`

*    Sort
    *   `City` in ascending order
    *   `createdHour` in ascending order

**Pseudocode:**
*    Need to find the biggest day
*    JOIN to the `data` table
*    produce `SELECT` statement
*    include `WHERE` statement
*    `GROUP BY`
*    `ORDER BY`

In [ ]:
def inner_join_example():

#     # display the list of cities
    query_inner_join = '''
                WITH topymd AS (
                                  SELECT strftime('%Y-%m-%d',CreatedDate) createdymd
                                          ,count(*) totalymd
                                  FROM data
                                  GROUP BY 1
                                  ORDER BY 2 DESC
                                  LIMIT 1
                                )
                SELECT a.city
                        ,strftime('%H',CreatedDate) createdhour
                        ,count(*) countoccur
                FROM data a
                    --this join gets the date with the most events from CTE
                    INNER JOIN topymd b 
                                on strftime('%Y-%m-%d',a.CreatedDate) = b.createdymd
                
                WHERE a.city IS NOT NULL
                GROUP BY 1,2
                ORDER BY 1,2
                '''
#     troubleshooting query
    # query_inner_join = '''
    #             --WITH topymd AS (
    #                               SELECT strftime('%Y-%m-%d',CreatedDate) createdymd
    #                                       ,count(*) totalymd
    #                               FROM data
    #                               GROUP BY 1
    #                               ORDER BY 2 DESC
    #                               LIMIT 1
    #             --                )
    #             /*
    #             SELECT a.city
    #                     ,strftime('%H',CreatedDate) createdhour
    #                     ,count(*) countoccur
    #             FROM data a
    #                 --this join gets the date with the most events from CTE
    #                 INNER JOIN topymd b 
    #                             on strftime('%Y-%m-%d',a.CreatedDate) = b.createdymd
                
    #             WHERE a.city IS NOT NULL
    #             GROUP BY 1,2
    #             ORDER BY 1,2
    #             */
    #             '''
    
    return query_inner_join

df_inner_join_example = pd.read_sql(inner_join_example(),conn_nyc)
display(df_inner_join_example)

### So what did we do here?

#### First the cte itself:

1. The CTE counted the number of rows (complaints) for each data.

2. The CTE sorted by the number in descending order.

3. The CTE returned one row, which is the date with the most complaints.

#### Next, in the main query:

1. The CTE inner joined to the main query on the date.

2. Because the join to the CTE is an inner join, it ensures that the only rows included/returned are those with that date.

### CTEs are fairly straightforward to troubleshoot.

For example, to ensure that the CTE is returning the correct date, all we have to do is comment out the non-CTE lines, and then execute the query.

Let's see how this is done. We will uncomment the troubleshooting query and discuss what we did.

### What are your questions so far?

### LEFT JOIN WITH SUBQUERY AND MULTIPLE CTEs

From the `data` table, for each `city`, return counts or a distribution of tickets per `hour` on the biggest day (by events) by `createdDate`. 

Hint, it's `2014-11-18` (8466 events) but how do we put this into code dynamically?

But this time, **only include `City` if they have an event on that day**. 

Additionally, `City` with `Hour` without events on that day should be included but should have a 0 for `countoccur`.

*    Columns
    *    `City`
    *    `createdHour`
    *    `countoccur`: the count of events. **REMEMBER: This should be 0 if there aren't any events for that city/hour**


*    Exclude NULL cities i.e. `WHERE city IS NOT NULL`


*    Sort
    *   `City` in ascending order
    *   `createdHour` in ascending order

**Pseudocode:**

*    Need to find the biggest day. This is query `b` below.

*    Need to get all the hours that occur on the biggest day. This is query `c` below.

*    Need to get all the cities/hour combinations possible for the biggest day. This is query `aa` below.

*    Need to get the counts for each city/hour on the biggest day, This is query `bb` below.

*    Need to join query `aa` with qery `bb` to retain all of query `aa`. This is this overall query, `a`.

*    produce `SELECT` statement

*    `GROUP BY`
*    `ORDER BY`

### Note that there are 5 distinct queries/subqueries:

a.  The base `data` table, or the main table the all of the other queries rely on.

b.  Gets the date with the most events. This CTE is written once and executed in multiple places. This is `CTE_topymd`.

c.  Gets the distinct hours on the date with the most events. This is `CTE_hours`. Depends on the date in `CTE_topymd`.

aa.  Gets a distinct list of city/createdhour for the date with the most events. This is `CTE_cityhours`. Depends on the date in `CTE_topymd`.

bb.  Gets the number of events for city/hour. SAME AS INNER JOIN EXAMPLE ABOVE. This is `CTE_cityhourcounts`. Depends on the date in `CTE_topymd`.

### CTE_topymd is defined once, and it is executed twice:

 - Once in the query CTE_city_hours
 
 - Once in the query CTE_cityhourcounts
 
### CTE_hours is defined once and it is executed 

- Once in the query CTE_cityhours

### CTE_cityhours is defined once and it is executed

- Once in the main query

### CTE_cityhourcounts is defined once and it is executed

- Once in the main query

In [ ]:
def left_join_example():

    # display the list of cities
    query_left_join = '''
                         WITH CTE_topymd AS ( --this gets the date with the most events
                                          SELECT strftime('%Y-%m-%d',CreatedDate) createdymd
                                                  ,count(*) totalymd
                                          FROM data
                                          GROUP BY 1
                                          ORDER BY 2 desc
                                          LIMIT 1
                                        ),
                           --independent query, does not rely on the one above
                           CTE_hours AS ( --this gets the distinct hours on the date with the most events
                                          SELECT distinct strftime('%Y-%m-%d',CreatedDate) createdymd
                                                          ,strftime('%H',CreatedDate) createdhour
                                          FROM data
                                        ),
                           --depends on the two CTEs above
                           CTE_cityhours AS ( --this gets a distinct list of city/createdhour for the date with the most events
                                          SELECT DISTINCT a.city
                                                          ,c.createdhour
                                          FROM data a
                                          INNER JOIN CTE_topymd b 
                                                      on strftime('%Y-%m-%d',a.CreatedDate) = b.createdymd
                                          LEFT JOIN CTE_hours c 
                                                      on strftime('%Y-%m-%d',a.CreatedDate) = c.createdymd
                                          WHERE a.city IS NOT NULL
                                        ),
                           --depends on the CTE_topymd above
                           CTE_cityhourcounts AS ( --this gets the number of events for city/hour. SAME AS JOIN EXAMPLE ABOVE
                                              SELECT a.city
                                                      ,strftime('%H',CreatedDate) createdhour
                                                      ,count(*) countoccur
                                              FROM data a
                                              INNER JOIN CTE_topymd b 
                                                      on strftime('%Y-%m-%d',a.CreatedDate) = b.createdymd
                                              WHERE a.city IS NOT NULL
                                              GROUP BY 1,2
                                        )
            --this is the main query
            SELECT 
                aa.city
                ,aa.createdhour
                ,COALESCE(bb.countoccur,0) countoccur
            
            FROM CTE_cityhours aa
                LEFT JOIN CTE_cityhourcounts bb 
                            ON aa.city=bb.city 
                            AND aa.createdhour=bb.createdhour
            ORDER BY 1,2
                '''
    return query_left_join

df_left_join_example = pd.read_sql(left_join_example(),conn_nyc)
display(df_left_join_example)

## Troubleshooting this query would be the same as above, and we leave the execution as an exercise for students.

## What questions do you have on CTEs?